In [ ]:
"""
===============================================================================
NEO4J KNOWLEDGE GRAPH IMPORT & ENTITY RESOLUTION PIPELINE
===============================================================================

PURPOSE:
This script imports legal case data from JSONL files into a Neo4j graph database
and resolves duplicate entities (people and organizations) to create a clean,
connected knowledge graph.

INPUT: 
- JSONL file where each line is a legal case with emails, people, organizations,
  drugs, documents, and other entities

OUTPUT:
- Neo4j graph database with nodes (Person, Organization, Email, RxNormDrug, etc.)
  and relationships (SENT, AFFILIATED_WITH, EMAIL_MENTIONS_DRUG, etc.)
- Duplicate entities merged into single nodes
- Statistics showing import success and entity resolution results

WORKFLOW:
1. Import all data from JSONL (creates duplicates)
2. Preview duplicate entities
3. Resolve duplicates by merging similar entities
4. Report statistics

===============================================================================
"""

from neo4j import GraphDatabase
import json, time, re, uuid
from typing import Any, Dict, List, Union
from difflib import SequenceMatcher
from collections import defaultdict

# =====================================================================
# SECTION 1: UTILITY FUNCTIONS
# =====================================================================

def ensure_list(x: Union[None, Dict[str, Any], List[Dict[str, Any]]]) -> List[Dict[str, Any]]:
    """
    Convert single items or None to lists for consistent processing.
    
    Args:
        x: None, a dict, or a list of dicts
        
    Returns:
        List of dicts (empty list if x is None)
        
    Example:
        ensure_list(None) -> []
        ensure_list({'name': 'John'}) -> [{'name': 'John'}]
        ensure_list([{'name': 'John'}]) -> [{'name': 'John'}]
    """
    if x is None:
        return []
    if isinstance(x, list):
        return x
    return [x]


# =====================================================================
# SECTION 2: CASE-LEVEL IMPORT FUNCTIONS
# =====================================================================

def upsert_case(tx, case_obj: Dict[str, Any]):
    """
    Create or update a Case node in Neo4j with all its connected entities.
    
    This is the top-level function that processes an entire legal case including:
    - Case metadata (identifier, legal status, dates)
    - Case-level entity mentions (topics, places)
    - All emails within the case
    
    Args:
        tx: Neo4j transaction object
        case_obj: Dictionary containing case data from JSONL
        
    Process:
        1. Create/update Case node with properties
        2. Process case-level mentions (topics, places)
        3. Recursively process all emails in the case
    """
    case_id = case_obj.get("identifier")
    if not case_id:
        return  # Skip if no case identifier

    # STEP 1: Create or update the Case node
    tx.run(
        """
        MERGE (c:Case {identifier: $identifier})
        SET
          c.semantic_type = $semantic_type,
          c.legalStatus = $legalStatus,
          c.dateFiled = $dateFiled,
          c.confidentialityNotice = $confidentialityNotice,
          c.language = $language
        """,
        identifier=case_id,
        semantic_type=case_obj.get("semantic_type"),
        legalStatus=case_obj.get("legalStatus"),
        dateFiled=case_obj.get("dateFiled"),
        confidentialityNotice=case_obj.get("confidentialityNotice"),
        language=case_obj.get("language"),
    )

    # STEP 2: Process case-level mentions (entities mentioned in case description)
    for mention in case_obj.get("mentions") or []:
        if isinstance(mention, dict):
            upsert_case_mention(tx, case_id, mention)

    # STEP 3: Process all emails that are part of this case
    for email_obj in ensure_list(case_obj.get("hasPart")):
        if isinstance(email_obj, dict):
            upsert_email_recursive(tx, case_id, email_obj, parent_email_id=None)


def upsert_case_mention(tx, case_id: str, mention: Dict[str, Any]):
    """
    Create entity nodes for things mentioned in the case description.
    
    Creates two types of entities:
    - Place nodes (for geographic locations like "California", "New York")
    - TopicEntity nodes (for topics like "drug safety", "clinical trials")
    
    Args:
        tx: Neo4j transaction
        case_id: Identifier of the parent case
        mention: Dictionary with entity information
        
    Creates:
        - Node: Place or TopicEntity
        - Relationship: (Case)-[:CASE_MENTIONS]->(Entity)
    """
    m_type = mention.get("@type")
    name = mention.get("name")
    if not name:
        return  # Skip if no name

    sem = mention.get("semantic_type")
    identifier = mention.get("identifier")

    # Determine node type based on entity type
    if m_type == "gpe":  # GPE = Geopolitical Entity (cities, states, countries)
        label = "Place"
    else:
        label = "TopicEntity"  # General topics, concepts, subjects

    # STEP 1: Create or update the entity node
    tx.run(
        f"""
        MERGE (m:{label} {{name: $name}})
        SET
          m.semantic_type = $semantic_type,
          m.identifier = $identifier
        """,
        name=name,
        semantic_type=sem,
        identifier=identifier,
    )

    # STEP 2: Create relationship from Case to this entity
    tx.run(
        f"""
        MATCH (c:Case {{identifier: $case_id}})
        MATCH (m:{label} {{name: $name}})
        MERGE (c)-[:CASE_MENTIONS]->(m)
        """,
        case_id=case_id,
        name=name,
    )


# =====================================================================
# SECTION 3: PERSON & ORGANIZATION IMPORT FUNCTIONS
# =====================================================================

def upsert_person(tx, person: Dict[str, Any]) -> str:
    """
    Create or update a Person node in Neo4j.
    
    KEY CONCEPT: Each person needs a unique identifier ("key") to avoid duplicates.
    The key is determined by:
    1. Email address (if available) - most reliable
    2. Name (if no email) - less reliable but better than nothing
    
    Args:
        tx: Neo4j transaction
        person: Dictionary with person data (name, email, affiliation)
        
    Returns:
        The unique key for this person (used to create relationships later)
        
    Creates:
        - Node: Person with properties (name, email, key)
        - Relationship: (Person)-[:AFFILIATED_WITH]->(Organization) if affiliation exists
    """
    if not person:
        return None

    # Extract person data
    name = person.get("name") or "Unknown"
    email_addr = person.get("email")
    sem = person.get("semantic_type")
    
    # CRITICAL: Determine unique key for this person
    # Priority: email > name (email is more unique)
    key = email_addr or name

    # STEP 1: Create or update Person node
    tx.run(
        """
        MERGE (p:Person {key: $key})
        SET
          p.name = $name,
          p.email = $email,
          p.semantic_type = $semantic_type
        """,
        key=key,
        name=name,
        email=email_addr,
        semantic_type=sem,
    )

    # STEP 2: If person has organizational affiliation, create that too
    aff = person.get("affiliation")
    if isinstance(aff, dict):
        upsert_org_for_person(tx, key, aff)

    return key  # Return key so caller can create relationships to this person


def upsert_org_for_person(tx, person_key: str, org: Dict[str, Any]):
    """
    Create Organization nodes and link them to a Person.
    
    Handles organizational hierarchy:
    - Creates the organization node
    - Links person to organization
    - If organization has a parent, creates that too
    - Links child organization to parent
    
    Args:
        tx: Neo4j transaction
        person_key: Unique key of the person
        org: Dictionary with organization data
        
    Creates:
        - Node: Organization
        - Node: Parent Organization (if exists)
        - Relationship: (Person)-[:AFFILIATED_WITH]->(Organization)
        - Relationship: (Organization)-[:SUBSIDIARY_OF]->(Parent)
        
    Example:
        Person "John Smith" works at "Pfizer Research"
        "Pfizer Research" is subsidiary of "Pfizer Inc"
        Creates: (John)-[:AFFILIATED_WITH]->(Pfizer Research)-[:SUBSIDIARY_OF]->(Pfizer Inc)
    """
    name = org.get("name")
    if not name:
        return

    role = org.get("role")  # e.g., "researcher", "executive"
    sem = org.get("semantic_type")

    # STEP 1: Create or update Organization node
    tx.run(
        """
        MERGE (o:Organization {name: $name})
        SET
          o.semantic_type = $semantic_type,
          o.role = $role
        """,
        name=name,
        semantic_type=sem,
        role=role,
    )

    # STEP 2: Link Person to Organization
    tx.run(
        """
        MATCH (p:Person {key: $person_key})
        MATCH (o:Organization {name: $name})
        MERGE (p)-[:AFFILIATED_WITH]->(o)
        """,
        person_key=person_key,
        name=name,
    )

    # STEP 3: Handle parent organization (if exists)
    parent = org.get("parentOrganization")
    if isinstance(parent, dict) and parent.get("name"):
        pname = parent.get("name")
        psem = parent.get("semantic_type")
        prole = parent.get("role")

        # Create parent organization node
        tx.run(
            """
            MERGE (po:Organization {name: $pname})
            SET
              po.semantic_type = $p_sem,
              po.role = $p_role
            """,
            pname=pname,
            p_sem=psem,
            p_role=prole,
        )

        # Link child to parent organization
        tx.run(
            """
            MATCH (o:Organization {name: $name})
            MATCH (po:Organization {name: $pname})
            MERGE (o)-[:SUBSIDIARY_OF]->(po)
            """,
            name=name,
            pname=pname,
        )


# =====================================================================
# SECTION 4: EMAIL & DOCUMENT IMPORT FUNCTIONS
# =====================================================================

def upsert_email_recursive(tx, case_id: str, email_obj: Dict[str, Any], parent_email_id: str = None):
    """
    Create Email nodes and all connected entities recursively.
    
    This is a RECURSIVE function because emails can contain forwarded emails,
    which can contain more forwarded emails, etc.
    
    Handles:
    - Email metadata (subject, date, importance, body)
    - Sender and recipients (creates Person nodes)
    - Entity mentions in email (drugs, topics, locations)
    - Documents attached to email
    - Forwarded/nested emails (recursive)
    - Email cross-references
    - RxNorm drug mentions
    - Enriched content (decisions, concerns, events, financials)
    
    Args:
        tx: Neo4j transaction
        case_id: Parent case identifier
        email_obj: Email data dictionary
        parent_email_id: If this is a forwarded email, ID of parent email
        
    Creates:
        - Node: Email
        - Nodes: Person (sender, recipients)
        - Nodes: RxNormDrug, TopicEntity, Location, Document, etc.
        - Many relationships connecting all these entities
    """
    if not email_obj:
        return

    # STEP 1: Get or generate email identifier
    email_id = email_obj.get("identifier") or email_obj.get("id")
    if not email_id:
        # Generate ID from subject and date if not provided
        email_id = f"{email_obj.get('subject', 'Unknown')}|{email_obj.get('dateSent', '')}"

    # STEP 2: Create or update Email node
    tx.run(
        """
        MERGE (e:Email {identifier: $identifier})
        SET
          e.semantic_type = $semantic_type,
          e.subject       = $subject,
          e.dateSent      = $dateSent,
          e.importance    = $importance,
          e.body          = $body
        """.strip(),
        identifier=email_id,
        semantic_type=email_obj.get("semantic_type"),
        subject=email_obj.get("subject"),
        dateSent=email_obj.get("dateSent"),
        importance=email_obj.get("importance"),
        body=email_obj.get("body"),
    )

    # STEP 3: Link Email to its Case
    if case_id:
        tx.run(
            """
            MATCH (c:Case {identifier: $case_id})
            MATCH (e:Email {identifier: $email_id})
            MERGE (c)-[:HAS_EMAIL]->(e)
            """.strip(),
            case_id=case_id,
            email_id=email_id,
        )

    # STEP 4: Handle email threading (forwarded messages)
    if parent_email_id:
        tx.run(
            """
            MATCH (parent:Email {identifier: $parent_id})
            MATCH (child:Email {identifier: $email_id})
            MERGE (parent)-[:FORWARDED_MESSAGE]->(child)
            """.strip(),
            parent_id=parent_email_id,
            email_id=email_id,
        )

    # STEP 5: Process sender
    sender = email_obj.get("sender")
    if isinstance(sender, dict):
        sender_key = upsert_person(tx, sender)  # Create Person node
        if sender_key:
            # Link Person to Email with SENT relationship
            tx.run(
                """
                MATCH (e:Email {identifier: $email_id})
                MATCH (p:Person {key: $sender_key})
                MERGE (p)-[:SENT]->(e)
                """.strip(),
                email_id=email_id,
                sender_key=sender_key,
            )

    # STEP 6: Process recipients (can be multiple)
    for rcpt in email_obj.get("recipient") or []:
        if isinstance(rcpt, dict):
            rcpt_key = upsert_person(tx, rcpt)  # Create Person node
            if rcpt_key:
                # Link Email to Person with SENT_TO relationship
                tx.run(
                    """
                    MATCH (e:Email {identifier: $email_id})
                    MATCH (p:Person {key: $rcpt_key})
                    MERGE (e)-[:SENT_TO]->(p)
                    """.strip(),
                    email_id=email_id,
                    rcpt_key=rcpt_key,
                )

    # STEP 7: Process entity mentions (topics, places mentioned in email)
    for mention in email_obj.get("mentions") or []:
        if isinstance(mention, dict):
            upsert_mention_for_email(tx, email_id, mention)

    # STEP 8: Process attachments (documents)
    for att in email_obj.get("attachments") or []:
        if isinstance(att, dict):
            upsert_attachment(tx, email_id, case_id, att)

    # STEP 9: Process RxNorm drugs mentioned in email
    for drug in email_obj.get("drugsRXnorm") or []:
        upsert_rxnorm_drug_for_email(tx, email_id, drug)

    # STEP 10: Process enriched content (AI-extracted information)
    enriched = email_obj.get("enriched_content") or {}
    if enriched:
        upsert_enriched_content_for_email(tx, email_id, enriched)

    # STEP 11: RECURSIVELY process forwarded/nested emails
    fwd = email_obj.get("forwardedMessage")
    if isinstance(fwd, dict):
        # Single forwarded email
        upsert_email_recursive(tx, case_id, fwd, parent_email_id=email_id)
    elif isinstance(fwd, list):
        # Multiple forwarded emails
        for child in fwd:
            if isinstance(child, dict):
                upsert_email_recursive(tx, case_id, child, parent_email_id=email_id)

    # STEP 12: Process email cross-references (emails that mention other emails)
    for me in email_obj.get("mentionsEmail") or []:
        if isinstance(me, dict):
            ref_id = me.get("identifier")
            if ref_id:
                upsert_cross_reference_email(
                    tx,
                    source_email_id=email_id,
                    target_email_id=ref_id,
                    similarity_score=None,
                )

    # STEP 13: Process cross-reference info (similar emails with scores)
    cross = email_obj.get("crossRefInfo") or {}
    for cref in cross.get("crossRefEmails") or []:
        if isinstance(cref, dict):
            cid = cref.get("cid")  # Email ID
            score = cref.get("score")  # Similarity score
            if cid:
                upsert_cross_reference_email(
                    tx,
                    source_email_id=email_id,
                    target_email_id=cid,
                    similarity_score=score,
                )


def upsert_mention_for_email(tx, email_id: str, mention: Dict[str, Any]):
    """
    Create entity nodes for things mentioned IN an email.
    
    Similar to upsert_case_mention but for email-level entities.
    
    Args:
        tx: Neo4j transaction
        email_id: Email that contains this mention
        mention: Entity data
        
    Creates:
        - Node: Place or TopicEntity
        - Relationship: (Email)-[:EMAIL_MENTIONS_PLACE or EMAIL_MENTIONS_TOPIC]->(Entity)
    """
    m_type = mention.get("@type")
    name = mention.get("name")
    if not name:
        return

    sem = mention.get("semantic_type")
    identifier = mention.get("identifier")
    role = mention.get("role")

    # Determine node type and relationship type
    if m_type == "gpe":
        label = "Place"
        rel_type = "EMAIL_MENTIONS_PLACE"
    else:
        label = "TopicEntity"
        rel_type = "EMAIL_MENTIONS_TOPIC"

    # Create entity node
    tx.run(
        f"""
        MERGE (m:{label} {{name: $name}})
        SET
          m.semantic_type = $semantic_type,
          m.identifier = $identifier,
          m.role = $role
        """,
        name=name,
        semantic_type=sem,
        identifier=identifier,
        role=role,
    )

    # Create relationship from email to entity
    tx.run(
        f"""
        MATCH (e:Email {{identifier: $email_id}})
        MATCH (m:{label} {{name: $name}})
        MERGE (e)-[:{rel_type}]->(m)
        """,
        email_id=email_id,
        name=name,
    )


def upsert_attachment(tx, email_id: str, case_id: str, attachment: Dict[str, Any]):
    """
    Create Document nodes for email attachments.
    
    Args:
        tx: Neo4j transaction
        email_id: Email that has this attachment
        case_id: Parent case
        attachment: Document data
        
    Creates:
        - Node: Document
        - Relationship: (Email)-[:HAS_ATTACHMENT]->(Document)
        - Relationship: (Case)-[:CASE_HAS_DOCUMENT]->(Document)
    """
    name = attachment.get("name")
    if not name:
        return

    sem = attachment.get("semantic_type")
    file_format = attachment.get("fileFormat")  # PDF, DOCX, XLSX, etc.
    desc = attachment.get("description")

    # Create Document node
    tx.run(
        """
        MERGE (d:Document {name: $name})
        SET
          d.semantic_type = $semantic_type,
          d.fileFormat = $fileFormat,
          d.description = $description
        """,
        name=name,
        semantic_type=sem,
        fileFormat=file_format,
        description=desc,
    )

    # Link Email to Document
    tx.run(
        """
        MATCH (e:Email {identifier: $email_id})
        MATCH (d:Document {name: $name})
        MERGE (e)-[:HAS_ATTACHMENT]->(d)
        """,
        email_id=email_id,
        name=name,
    )

    # Link Case to Document
    if case_id:
        tx.run(
            """
            MATCH (c:Case {identifier: $case_id})
            MATCH (d:Document {name: $name})
            MERGE (c)-[:CASE_HAS_DOCUMENT]->(d)
            """,
            case_id=case_id,
            name=name,
        )


# =====================================================================
# SECTION 5: SPECIALIZED ENTITY IMPORT FUNCTIONS
# =====================================================================

def upsert_rxnorm_drug_for_email(tx, email_id: str, drug):
    """
    Create RxNormDrug nodes for pharmaceutical drugs mentioned in emails.
    
    RxNorm is a standardized drug naming system. This function handles
    drug data that can come in different formats (string or dict).
    
    Args:
        tx: Neo4j transaction
        email_id: Email mentioning this drug
        drug: Can be a string ("Oxycodone") or dict with details
        
    Creates:
        - Node: RxNormDrug
        - Relationship: (Email)-[:EMAIL_MENTIONS_DRUG]->(RxNormDrug)
        
    Example:
        Input: "Oxycodone"
        Creates: (RxNormDrug {name: "Oxycodone"})
    """
    if drug is None:
        return

    # Handle different input formats
    if isinstance(drug, str):
        name = drug.strip()
        rxcui = None  # RxNorm Concept Unique Identifier
        source = None
    elif isinstance(drug, dict):
        name = (drug.get("name") or drug.get("drug_name") or drug.get("id") or "").strip()
        rxcui = drug.get("rxcui") or drug.get("rxnorm_id")
        source = drug.get("source") or drug.get("origin") or "RxNorm"
    else:
        return

    if not name:
        return

    # Create drug node
    tx.run(
        """
        MERGE (d:RxNormDrug {name: $name})
        SET
          d.rxnorm_id = coalesce(d.rxnorm_id, $rxcui),
          d.source    = coalesce(d.source, $source)
        """.strip(),
        name=name,
        rxcui=rxcui,
        source=source,
    )

    # Link Email to Drug
    tx.run(
        """
        MATCH (e:Email {identifier: $email_id})
        MATCH (d:RxNormDrug {name: $name})
        MERGE (e)-[:EMAIL_MENTIONS_DRUG]->(d)
        """.strip(),
        email_id=email_id,
        name=name,
    )


def upsert_enriched_content_for_email(tx, email_id: str, enriched):
    """
    Create nodes for AI-extracted/enriched content from emails.
    
    This processes additional information extracted by AI/NLP from emails:
    - Decisions made
    - Concerns raised
    - Events mentioned
    - Financial information
    - Locations mentioned
    - People mentioned (beyond sender/recipient)
    
    Args:
        tx: Neo4j transaction
        email_id: Email containing this enriched content
        enriched: Dictionary with enriched data
        
    Creates multiple node types:
        - Decision nodes: Important decisions mentioned
        - Concern nodes: Issues/concerns raised
        - Event nodes: Events referenced
        - Financial nodes: Financial mentions
        - Location nodes: Geographic locations
        - Additional Person nodes: People mentioned in body
    """
    if not enriched:
        return

    def _create_text_nodes(label: str, rel_type: str, items):
        """
        Helper function to create nodes for text-based enriched content.
        
        Args:
            label: Node label (Decision, Concern, Event, Financial)
            rel_type: Relationship type
            items: List of items to process
        """
        for item in items or []:
            if item is None:
                continue
            if isinstance(item, dict):
                text = (item.get("text") or item.get("value") or "").strip()
                source = item.get("source")
            else:
                text = str(item).strip()
                source = None
            if not text:
                continue
            
            # Create node and link to email
            tx.run(
                f"""
                MATCH (e:Email {{identifier: $email_id}})
                CREATE (n:{label} {{text: $text, source: $source}})
                MERGE (e)-[:{rel_type}]->(n)
                """.strip(),
                email_id=email_id,
                text=text,
                source=source,
            )

    # Process different types of enriched content
    _create_text_nodes("Decision",  "HAS_DECISION",   enriched.get("decisions_made"))
    _create_text_nodes("Concern",   "HAS_CONCERN",    enriched.get("concerns_raised"))
    _create_text_nodes("Event",     "HAS_EVENT",      enriched.get("events_mentioned"))
    _create_text_nodes("Financial", "HAS_FINANCIAL",  enriched.get("financial_mentions"))

    # Process locations
    for loc in enriched.get("locations_mentioned") or []:
        if loc is None:
            continue
        if isinstance(loc, dict):
            name = (loc.get("name") or "").strip()
            source = loc.get("source")
        else:
            name = str(loc).strip()
            source = None
        if not name:
            continue
        
        # Create Location node
        tx.run(
            """
            MERGE (l:Location {name: $name})
            SET l.source = coalesce(l.source, $source)
            """.strip(),
            name=name,
            source=source,
        )
        
        # Link Email to Location
        tx.run(
            """
            MATCH (e:Email {identifier: $email_id})
            MATCH (l:Location {name: $name})
            MERGE (e)-[:EMAIL_MENTIONS_LOCATION]->(l)
            """.strip(),
            email_id=email_id,
            name=name,
        )

    # Process people mentioned (beyond sender/recipient)
    for pm in enriched.get("people_mentioned") or []:
        if pm is None:
            continue
        if isinstance(pm, dict):
            name = (pm.get("name") or "").strip()
            email_addr = pm.get("email") or None
        else:
            name = str(pm).strip()
            email_addr = None
        if not name:
            continue

        # Create Person node
        person_dict = {"name": name, "email": email_addr, "semantic_type": "Person"}
        person_key = upsert_person(tx, person_dict)
        if person_key:
            # Link Email to Person (different from SENT/SENT_TO)
            tx.run(
                """
                MATCH (p:Person {key: $key})
                MATCH (e:Email {identifier: $email_id})
                MERGE (e)-[:MENTIONS_PERSON_ENRICHED]->(p)
                """.strip(),
                key=person_key,
                email_id=email_id,
            )


def upsert_cross_reference_email(tx, source_email_id: str, target_email_id: str, similarity_score: float = None):
    """
    Create relationships between emails that reference each other.
    
    This handles email threading and cross-references with similarity scores.
    
    Args:
        tx: Neo4j transaction
        source_email_id: Email that references another
        target_email_id: Email being referenced
        similarity_score: Optional similarity score (0-1) between emails
        
    Creates:
        - Nodes: Both emails (if they don't exist)
        - Relationship: (Email)-[:REFERS_TO_EMAIL {similarity_score}]->(Email)
        
    Example:
        Email A says "As discussed in email B..."
        Creates: (Email A)-[:REFERS_TO_EMAIL]->(Email B)
    """
    if not source_email_id or not target_email_id:
        return

    tx.run(
        """
        MERGE (src:Email {identifier: $source_id})
        MERGE (tgt:Email {identifier: $target_id})
        MERGE (src)-[r:REFERS_TO_EMAIL]->(tgt)
        SET r.similarity_score = $similarity_score
        """.strip(),
        source_id=source_email_id,
        target_id=target_email_id,
        similarity_score=similarity_score,
    )


# =====================================================================
# SECTION 6: MAIN IMPORT FUNCTION
# =====================================================================

def import_jsonl_to_neo4j(
    jsonl_path: str,
    uri: str,
    user: str,
    password: str,
    log_every: int = 25,
):
    """
    Main import function: Read JSONL file and import all data to Neo4j.
    
    JSONL Format: Each line is a JSON object representing one legal case
    with nested emails, people, organizations, etc.
    
    Args:
        jsonl_path: Path to JSONL file
        uri: Neo4j connection URI (e.g., "neo4j://localhost:7687")
        user: Neo4j username
        password: Neo4j password
        log_every: Print progress every N lines
        
    Process:
        1. Connect to Neo4j
        2. Read JSONL file line by line
        3. Parse each line as a case
        4. Import case and all nested entities
        5. Handle errors gracefully (continue on failure)
        6. Report statistics
        
    Features:
        - Progress logging
        - Error handling per line (one bad line doesn't kill whole import)
        - Statistics tracking (success, failure, skipped)
        - Performance timing
    """
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Statistics counters
    total_lines = 0
    success_cases = 0
    skipped_lines = 0
    failed_cases = 0

    with driver.session() as session:
        with open(jsonl_path, "r", encoding="utf-8") as f:
            start_time = time.time()
            
            # Process each line in JSONL file
            for line_no, line in enumerate(f, start=1):
                total_lines += 1
                line = line.strip()
                
                # Skip empty lines
                if not line:
                    skipped_lines += 1
                    continue

                # Progress logging
                if line_no % log_every == 0:
                    print(f"[INFO] Processing line {line_no}... (success={success_cases}, failed={failed_cases}, skipped={skipped_lines})")
                    print('\t took', time.time() - start_time, 'seconds')

                # STEP 1: Parse the JSON wrapper
                try:
                    wrapper = json.loads(line)
                except json.JSONDecodeError as e:
                    print(f"[WARN] Skipping line {line_no}: invalid JSON wrapper ({e})")
                    skipped_lines += 1
                    continue

                # STEP 2: Extract the 'output' field (contains actual case data)
                output_raw = wrapper.get("output")
                if not output_raw:
                    print(f"[WARN] Skipping line {line_no}: no 'output' field")
                    skipped_lines += 1
                    continue

                # STEP 3: Parse the case object
                try:
                    case_obj = json.loads(output_raw)
                except json.JSONDecodeError:
                    # Sometimes output is already a dict, not a string
                    if isinstance(output_raw, dict):
                        case_obj = output_raw
                    else:
                        print(f"[WARN] Skipping line {line_no}: invalid 'output' JSON")
                        skipped_lines += 1
                        continue

                case_id = case_obj.get("identifier")

                # STEP 4: Import this case to Neo4j (with error handling)
                try:
                    def work(tx):
                        upsert_case(tx, case_obj)

                    session.execute_write(work)
                    success_cases += 1

                except Exception as e:
                    # Don't let one bad case kill the whole import
                    failed_cases += 1
                    print(f"[ERROR] Failed to import case on line {line_no} (case_id={case_id!r}): {type(e).__name__}: {e}")

    driver.close()

    # Print final statistics
    print("\n=== Import summary ===")
    print(f"Total lines read:     {total_lines}")
    print(f"Successful cases:     {success_cases}")
    print(f"Failed cases:         {failed_cases}")
    print(f"Skipped lines:        {skipped_lines}")
    print('Runtime (s):          ', time.time() - start_time)


# =====================================================================
# SECTION 7: ENTITY RESOLUTION - STATISTICS CLASS
# =====================================================================

class GraphStats:
    """
    Helper class to collect and display graph statistics.
    
    Used to measure the impact of entity resolution by comparing
    graph state before and after merging duplicates.
    """
    
    @staticmethod
    def get_stats(driver) -> Dict[str, Any]:
        """
        Collect comprehensive statistics about the graph.
        
        Queries Neo4j to count:
        - Nodes by type (Person, Email, Organization, etc.)
        - Relationships by type (SENT, AFFILIATED_WITH, etc.)
        - Specific details about Persons and Organizations
        
        Args:
            driver: Neo4j driver instance
            
        Returns:
            Dictionary with all statistics
        """
        stats = {}
        
        with driver.session() as session:
            # Count nodes by type
            result = session.run("""
                MATCH (n)
                RETURN labels(n)[0] as label, count(*) as count
                ORDER BY count DESC
            """)
            node_counts = {record['label']: record['count'] for record in result}
            stats['nodes'] = node_counts
            stats['total_nodes'] = sum(node_counts.values())
            
            # Count relationships by type
            result = session.run("""
                MATCH ()-[r]->()
                RETURN type(r) as rel_type, count(*) as count
                ORDER BY count DESC
            """)
            rel_counts = {record['rel_type']: record['count'] for record in result}
            stats['relationships'] = rel_counts
            stats['total_relationships'] = sum(rel_counts.values())
            
            # Detailed person statistics
            result = session.run("""
                MATCH (p:Person)
                WITH count(p) as total,
                     sum(CASE WHEN p.email IS NOT NULL THEN 1 ELSE 0 END) as with_email,
                     sum(CASE WHEN p.email IS NULL THEN 1 ELSE 0 END) as without_email
                RETURN total, with_email, without_email
            """)
            person_stats = result.single()
            if person_stats:
                stats['persons'] = {
                    'total': person_stats['total'],
                    'with_email': person_stats['with_email'],
                    'without_email': person_stats['without_email']
                }
            
            # Count organizations
            result = session.run("MATCH (o:Organization) RETURN count(o) as count")
            stats['organizations'] = result.single()['count']
            
            # Count emails
            result = session.run("MATCH (e:Email) RETURN count(e) as count")
            stats['emails'] = result.single()['count']
            
        return stats
    
    @staticmethod
    def print_stats(stats: Dict[str, Any], title: str = "Graph Statistics"):
        """Print statistics in a formatted, readable way."""
        print("\n" + "=" * 70)
        print(f"{title:^70}")
        print("=" * 70)
        
        print(f"\n📊 OVERALL:")
        print(f"  Total Nodes:         {stats['total_nodes']:,}")
        print(f"  Total Relationships: {stats['total_relationships']:,}")
        
        print(f"\n📦 NODES BY TYPE:")
        for label, count in sorted(stats['nodes'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {label:20s} {count:,}")
        
        print(f"\n🔗 RELATIONSHIPS BY TYPE:")
        for rel_type, count in sorted(stats['relationships'].items(), key=lambda x: x[1], reverse=True):
            print(f"  {rel_type:25s} {count:,}")
        
        if 'persons' in stats:
            print(f"\n👥 PERSON DETAILS:")
            print(f"  Total Persons:       {stats['persons']['total']:,}")
            print(f"  With Email:          {stats['persons']['with_email']:,}")
            print(f"  Without Email:       {stats['persons']['without_email']:,}")
        
        print(f"\n📧 OTHER:")
        print(f"  Organizations:       {stats.get('organizations', 0):,}")
        print(f"  Emails:              {stats.get('emails', 0):,}")
        
        print("=" * 70)
    
    @staticmethod
    def print_comparison(before: Dict[str, Any], after: Dict[str, Any]):
        """
        Print before/after comparison showing impact of entity resolution.
        
        Shows how many nodes were merged and percentage reduction.
        """
        print("\n" + "=" * 70)
        print(f"{'BEFORE vs AFTER ENTITY RESOLUTION':^70}")
        print("=" * 70)
        
        node_reduction = before['total_nodes'] - after['total_nodes']
        node_reduction_pct = (node_reduction / before['total_nodes'] * 100) if before['total_nodes'] > 0 else 0
        
        print(f"\n📊 OVERALL CHANGES:")
        print(f"  Nodes:         {before['total_nodes']:,} → {after['total_nodes']:,} ({node_reduction:+,} nodes, {node_reduction_pct:.1f}% reduction)")
        print(f"  Relationships: {before['total_relationships']:,} → {after['total_relationships']:,}")
        
        print(f"\n📦 NODE CHANGES BY TYPE:")
        all_labels = set(before['nodes'].keys()) | set(after['nodes'].keys())
        for label in sorted(all_labels):
            before_count = before['nodes'].get(label, 0)
            after_count = after['nodes'].get(label, 0)
            change = after_count - before_count
            if change != 0:
                print(f"  {label:20s} {before_count:,} → {after_count:,} ({change:+,})")
        
        if 'persons' in before and 'persons' in after:
            print(f"\n👥 PERSON CHANGES:")
            before_persons = before['persons']['total']
            after_persons = after['persons']['total']
            merged = before_persons - after_persons
            print(f"  Total:       {before_persons:,} → {after_persons:,}")
            print(f"  Merged:      {merged:,} duplicate(s) removed")
            if merged > 0:
                print(f"  Reduction:   {merged/before_persons*100:.1f}% of persons were duplicates")
        
        print("=" * 70)


# =====================================================================
# SECTION 8: ENTITY RESOLUTION - RESOLVER CLASS
# =====================================================================

class EntityResolver:
    """
    Main class for entity resolution (merging duplicate nodes).
    
    PROBLEM: During import, the same person or organization might be created
    multiple times with slight variations in names:
    - "John Smith" vs "J. Smith" vs "Smith, John"
    - "Microsoft Corp" vs "Microsoft Corporation" vs "Microsoft"
    
    SOLUTION: This class finds and merges similar entities based on:
    - String similarity matching
    - Email address matching (for persons)
    - Normalized name comparison
    
    APPROACH:
    1. Find duplicates using similarity thresholds
    2. Preview what would be merged (optional)
    3. Merge duplicate nodes by transferring relationships
    4. Track statistics before/after
    """
    
    def __init__(self, uri: str, user: str, password: str):
        """Initialize connection to Neo4j."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.stats_before = None
        self.stats_after = None
    
    def close(self):
        """Close Neo4j connection."""
        self.driver.close()
    
    @staticmethod
    def normalize_string(s: str) -> str:
        """
        Normalize string for comparison.
        
        Removes noise that prevents matching:
        - Converts to lowercase
        - Removes punctuation
        - Removes extra spaces
        - Removes common company suffixes (Inc, LLC, Corp, etc.)
        
        Example:
            "Microsoft Corporation" -> "microsoft"
            "Smith, John Jr." -> "smith john jr"
        """
        if not s:
            return ""
        s = s.lower().strip()
        s = re.sub(r'[^\w\s]', ' ', s)  # Remove punctuation
        s = re.sub(r'\s+', ' ', s).strip()  # Collapse spaces
        
        # Remove common company suffixes
        suffixes = [' inc', ' llc', ' lp', ' llp', ' corp', ' corporation', ' co', ' ltd', ' plc']
        for suffix in suffixes:
            if s.endswith(suffix):
                s = s[:-len(suffix)].strip()
        return s
        
    @staticmethod
    def similarity_score(s1: str, s2: str) -> float:
        """
        Calculate similarity between two strings (0-1 scale).
        
        Uses SequenceMatcher algorithm which compares character sequences.
        
        Returns:
            0.0 = completely different
            1.0 = identical
            0.9 = very similar (might be duplicate)
            
        Example:
            similarity("John Smith", "J. Smith") -> 0.72
            similarity("Microsoft", "Microsoft Corp") -> 0.85
        """
        s1_norm = EntityResolver.normalize_string(s1)
        s2_norm = EntityResolver.normalize_string(s2)
        return SequenceMatcher(None, s1_norm, s2_norm).ratio()
    
    def find_duplicate_persons(self, threshold: float = 0.9) -> List[tuple]:
        """
        Find potential duplicate person nodes using intelligent blocking.
        
        CHALLENGE: Comparing every person with every other person is O(n²)
        which is too slow for thousands of people.
        
        SOLUTION: Use "blocking" - group similar people together first,
        then only compare within groups. Groups are based on:
        - Email domain (people@company.com)
        - First word of name (all "John" names compared together)
        
        Args:
            threshold: Similarity score threshold (0-1) for considering duplicates
            
        Returns:
            List of tuples: (person1_key, person2_key, similarity_score)
            
        Algorithm:
        1. Fetch all persons from database
        2. Create blocks (groups) by email domain and name
        3. Compare persons only within same block
        4. Return pairs with similarity >= threshold
        """
        duplicates = []
        
        with self.driver.session() as session:
            # Fetch all persons with their organizations
            result = session.run("""
                MATCH (p:Person)
                OPTIONAL MATCH (p)-[:AFFILIATED_WITH]->(o:Organization)
                RETURN p.key as key, p.name as name, p.email as email, 
                       collect(DISTINCT o.name) as orgs
            """)
            
            persons = list(result)
            
            # Create blocks for efficient comparison
            email_blocks = defaultdict(list)
            name_blocks = defaultdict(list)
            
            for p in persons:
                # Skip invalid persons
                if not p['key'] or p['key'] == 'None' or not isinstance(p['key'], str):
                    continue
                if p['key'].startswith('unknown:'):
                    continue
                    
                # Block by email domain
                if p.get('email') and '@' in p['email']:
                    domain = p['email'].split('@')[-1].lower()
                    if domain:
                        email_blocks[domain].append(p)
                
                # Block by first word of name
                if p.get('name') and p['name'].lower() not in ['unknown', '[unknown]']:
                    normalized = self.normalize_string(p['name'])
                    if normalized:
                        first_word = normalized.split()[0] if normalized.split() else ''
                        if first_word and len(first_word) > 2:
                            name_blocks[first_word].append(p)
            
            # Compare within blocks only
            seen = set()
            
            # Process email blocks (high confidence)
            for block in email_blocks.values():
                for i, p1 in enumerate(block):
                    for p2 in block[i+1:]:
                        pair = tuple(sorted([p1['key'], p2['key']]))
                        if pair in seen:
                            continue
                        seen.add(pair)
                        
                        # Exact email match = definite duplicate
                        if p1.get('email') and p2.get('email'):
                            if p1['email'].lower() == p2['email'].lower():
                                duplicates.append((p1['key'], p2['key'], 1.0))
                                continue
                        
                        # Name similarity check
                        if p1.get('name') and p2.get('name'):
                            name_sim = self.similarity_score(p1['name'], p2['name'])
                            if name_sim >= threshold:
                                duplicates.append((p1['key'], p2['key'], name_sim))
            
            # Process name blocks (lower confidence, higher threshold)
            for block in name_blocks.values():
                for i, p1 in enumerate(block):
                    for p2 in block[i+1:]:
                        pair = tuple(sorted([p1['key'], p2['key']]))
                        if pair in seen:
                            continue
                        seen.add(pair)
                        
                        # Stricter threshold for name-only matches
                        name_sim = self.similarity_score(p1.get('name', ''), p2.get('name', ''))
                        if name_sim >= min(threshold + 0.05, 0.98):
                            duplicates.append((p1['key'], p2['key'], name_sim))
        
        return duplicates
    
    def merge_persons(self, keep_key: str, merge_key: str):
        """
        Merge two person nodes into one.
        
        Process:
        1. Validate keys (no empty, None, or self-merges)
        2. Transfer all relationships from merge_key to keep_key
        3. Merge properties (keep best available data)
        4. Delete the merged node
        
        Args:
            keep_key: Key of person to keep
            merge_key: Key of person to merge (will be deleted)
        """
        # Validation
        if not keep_key or not merge_key:
            print(f"  ✗ Skipping invalid merge: empty key")
            return
        if str(keep_key) == 'None' or str(merge_key) == 'None':
            print(f"  ✗ Skipping invalid merge: None key")
            return
        if keep_key == merge_key:
            print(f"  ✗ Skipping self-merge: {keep_key}")
            return
        if keep_key.startswith('unknown:') or merge_key.startswith('unknown:'):
            print(f"  ✗ Skipping auto-generated unknown key merge")
            return
        
        with self.driver.session() as session:
            session.execute_write(self._do_merge_persons, keep_key, merge_key)
    
    @staticmethod
    def _do_merge_persons(tx, keep_key: str, merge_key: str):
        """
        Transaction function to perform the actual person merge.
        
        Transfers all relationships and properties from merge node to keep node.
        """
        # Transfer SENT relationships (emails sent by this person)
        tx.run("""
            MATCH (merge:Person {key: $merge_key})-[r:SENT]->(e:Email)
            MATCH (keep:Person {key: $keep_key})
            MERGE (keep)-[:SENT]->(e)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Transfer SENT_TO relationships (emails received by this person)
        tx.run("""
            MATCH (e:Email)-[r:SENT_TO]->(merge:Person {key: $merge_key})
            MATCH (keep:Person {keep_key: $keep_key})
            MERGE (e)-[:SENT_TO]->(keep)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Transfer AFFILIATED_WITH relationships
        tx.run("""
            MATCH (merge:Person {key: $merge_key})-[r:AFFILIATED_WITH]->(o:Organization)
            MATCH (keep:Person {key: $keep_key})
            MERGE (keep)-[:AFFILIATED_WITH]->(o)
            DELETE r
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Merge properties (keep best available)
        tx.run("""
            MATCH (merge:Person {key: $merge_key})
            MATCH (keep:Person {key: $keep_key})
            SET keep.email = COALESCE(keep.email, merge.email),
                keep.name = CASE 
                    WHEN keep.name IN ['Unknown', '[unknown]', ''] THEN merge.name
                    ELSE keep.name
                END,
                keep.semantic_type = COALESCE(keep.semantic_type, merge.semantic_type)
        """, keep_key=keep_key, merge_key=merge_key)
        
        # Delete the merged node
        tx.run("MATCH (p:Person {key: $merge_key}) DETACH DELETE p", merge_key=merge_key)
    
    def find_duplicate_organizations(self, threshold: float = 0.85) -> List[tuple]:
        """
        Find duplicate organization nodes by name similarity.
        
        Similar to find_duplicate_persons but simpler since organizations
        only have names (no email addresses).
        
        Returns:
            List of tuples: (org1_name, org2_name, similarity_score)
        """
        duplicates = []

        with self.driver.session() as session:
            result = session.run("MATCH (o:Organization) RETURN o.name as name")
            orgs = list(result)

            # Normalize all organization names
            normalized_map = {}
            for org in orgs:
                if not org['name']:
                    continue
                normalized = self.normalize_string(org['name'])
                if normalized:
                    normalized_map[org['name']] = normalized

            # Compare normalized names
            seen = set()
            names = list(normalized_map.keys())
            for i, o1 in enumerate(names):
                for o2 in names[i+1:]:
                    n1 = normalized_map[o1]
                    n2 = normalized_map[o2]
                    pair = tuple(sorted([n1, n2]))
                    if pair in seen:
                        continue
                    seen.add(pair)

                    score = SequenceMatcher(None, n1, n2).ratio()
                    if score >= threshold:
                        # Always merge into alphabetically first
                        keep = min(n1, n2)
                        merge = max(n1, n2)
                        duplicates.append((keep, merge, score))

        return duplicates

    def merge_organizations(self, keep_norm: str, merge_norm: str):
        """
        Merge two organization nodes.
        
        Similar process to merge_persons but for organizations.
        """
        # Convert normalized names back to actual node names
        with self.driver.session() as session:
            keep_node = session.run(
                "MATCH (o:Organization) WHERE toLower(o.name) CONTAINS $norm RETURN o.name as name",
                norm=keep_norm
            ).single()
            merge_node = session.run(
                "MATCH (o:Organization) WHERE toLower(o.name) CONTAINS $norm RETURN o.name as name",
                norm=merge_norm
            ).single()

        if keep_node and merge_node:
            self._do_merge_orgs(self.driver.session(), keep_node['name'], merge_node['name'])

    @staticmethod
    def _do_merge_orgs(tx, keep_name: str, merge_name: str):
        """Transaction function to merge organization nodes."""
        # Transfer all relationship types that organizations have
        tx.run("""
            MATCH (p:Person)-[r:AFFILIATED_WITH]->(merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            MERGE (p)-[:AFFILIATED_WITH]->(keep)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        tx.run("""
            MATCH (sub:Organization)-[r:SUBSIDIARY_OF]->(merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            MERGE (sub)-[:SUBSIDIARY_OF]->(keep)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            OPTIONAL MATCH (merge)-[r:SUBSIDIARY_OF]->(parent:Organization)
            WITH keep, parent, r
            WHERE parent IS NOT NULL
            MERGE (keep)-[:SUBSIDIARY_OF]->(parent)
            DELETE r
        """, keep_name=keep_name, merge_name=merge_name)
        
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            MATCH (keep:Organization {name: $keep_name})
            SET keep.semantic_type = COALESCE(keep.semantic_type, merge.semantic_type),
                keep.role = COALESCE(keep.role, merge.role)
        """, keep_name=keep_name, merge_name=merge_name)
        
        tx.run("""
            MATCH (merge:Organization {name: $merge_name})
            DETACH DELETE merge
        """, merge_name=merge_name)
    def preview_duplicates(self, person_threshold: float = 0.9, org_threshold: float = 0.85):
        """Preview what duplicates would be found without merging"""
        print("\n" + "=" * 70)
        print(f"{'DUPLICATE PREVIEW (No Merging)':^70}")
        print("=" * 70)
        
        # Get current stats
        stats = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(stats, "CURRENT GRAPH STATE")
        
        # Preview person duplicates
        print(f"\n👥 POTENTIAL PERSON DUPLICATES (threshold={person_threshold}):")
        person_dupes = self.find_duplicate_persons(person_threshold)
        
        if not person_dupes:
            print("  ✓ No person duplicates found!")
        else:
            print(f"  Found {len(person_dupes)} potential duplicate pair(s):\n")
            
            # Fetch details for better display
            with self.driver.session() as session:
                shown = 0
                for i, (key1, key2, score) in enumerate(person_dupes, 1):
                    if shown >= 10:  # Only show first 10
                        break
                    
                    # Check if both persons still exist
                    p1 = session.run(
                        "MATCH (p:Person {key: $key}) RETURN p.name as name, p.email as email",
                        key=key1
                    ).single()
                    p2 = session.run(
                        "MATCH (p:Person {key: $key}) RETURN p.name as name, p.email as email",
                        key=key2
                    ).single()
                    
                    # Skip if either person was already deleted
                    if not p1 or not p2:
                        continue
                    
                    shown += 1
                    print(f"  [{shown}] Similarity: {score:.3f}")
                    print(f"      Person A: {p1['name']} ({p1['email'] or 'no email'})")
                    print(f"      Person B: {p2['name']} ({p2['email'] or 'no email'})")
                    print()
            
            if len(person_dupes) > 10:
                print(f"  ... and {len(person_dupes) - 10} more duplicate pair(s)")
        
        # Preview organization duplicates
        print(f"\n🏢 POTENTIAL ORGANIZATION DUPLICATES (threshold={org_threshold}):")
        org_dupes = self.find_duplicate_organizations(org_threshold)
        
        if not org_dupes:
            print("  ✓ No organization duplicates found!")
        else:
            print(f"  Found {len(org_dupes)} potential duplicate pair(s):\n")
            
            with self.driver.session() as session:
                shown = 0
                for i, (name1, name2, score) in enumerate(org_dupes, 1):
                    if shown >= 10:
                        break
                    
                    # Check if both orgs still exist
                    o1 = session.run(
                        "MATCH (o:Organization {name: $name}) RETURN o.name as name",
                        name=name1
                    ).single()
                    o2 = session.run(
                        "MATCH (o:Organization {name: $name}) RETURN o.name as name",
                        name=name2
                    ).single()
                    
                    if not o1 or not o2:
                        continue
                    
                    shown += 1
                    print(f"  [{shown}] Similarity: {score:.3f}")
                    print(f"      Org A: {name1}")
                    print(f"      Org B: {name2}")
                    print()
            
            if len(org_dupes) > 10:
                print(f"  ... and {len(org_dupes) - 10} more duplicate pair(s)")
        
        # Summary
        total_potential_merges = len(person_dupes) + len(org_dupes)
        print("\n" + "=" * 70)
        print(f"SUMMARY: {total_potential_merges} potential merge(s) identified")
        print(f"  • {len(person_dupes)} person merge(s)")
        print(f"  • {len(org_dupes)} organization merge(s)")
        
        if total_potential_merges > 0:
            estimated_reduction = len(person_dupes) + len(org_dupes)
            print(f"\nEstimated node reduction: {estimated_reduction} nodes")
        
        print("=" * 70)
    def resolve_all(self, person_threshold: float = 0.95, org_threshold: float = 0.90):
        """
        Run complete entity resolution process.
        
        Args:
            person_threshold: Similarity threshold for persons (0-1)
            org_threshold: Similarity threshold for organizations (0-1)
            
        Process:
        1. Capture BEFORE statistics
        2. Find and merge duplicate persons
        3. Find and merge duplicate organizations
        4. Capture AFTER statistics
        5. Display comparison
        """
        print("\n" + "=" * 70)
        print(f"{'ENTITY RESOLUTION STARTING':^70}")
        print("=" * 70)
        
        # BEFORE stats
        print("\n[1/4] Capturing BEFORE statistics...")
        self.stats_before = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(self.stats_before, "BEFORE ENTITY RESOLUTION")
        
        start_time = time.time()
        
        # Resolve Persons
        print(f"\n[2/4] Resolving Persons (threshold={person_threshold})...")
        person_dupes = self.find_duplicate_persons(person_threshold)
        print(f"  Found {len(person_dupes)} potential person duplicates")
        
        person_merge_count = 0
        person_skip_count = 0
        for keep, merge, score in person_dupes:
            try:
                if not keep or not merge or keep == merge:
                    person_skip_count += 1
                    continue
                if str(keep) == 'None' or str(merge) == 'None':
                    person_skip_count += 1
                    continue
                
                print(f"    Merging {merge} -> {keep} (similarity: {score:.3f})")
                self.merge_persons(keep, merge)
                person_merge_count += 1
            except Exception as e:
                print(f"    ✗ Failed to merge {merge} -> {keep}: {e}")
                person_skip_count += 1
        
        print(f"  ✓ Successfully merged {person_merge_count} person node(s)")
        if person_skip_count > 0:
            print(f"  ⚠ Skipped {person_skip_count} invalid merge(s)")
        
        # Resolve Organizations
        print(f"\n[3/4] Resolving Organizations (threshold={org_threshold})...")
        org_dupes = self.find_duplicate_organizations(org_threshold)
        print(f"  Found {len(org_dupes)} potential organization duplicates")
        
        org_merge_count = 0
        org_skip_count = 0
        for keep, merge, score in org_dupes:
            try:
                if not keep or not merge or keep == merge:
                    org_skip_count += 1
                    continue
                
                print(f"    Merging '{merge}' -> '{keep}' (similarity: {score:.3f})")
                self.merge_organizations(keep, merge)
                org_merge_count += 1
            except Exception as e:
                print(f"    ✗ Failed to merge '{merge}' -> '{keep}': {e}")
                org_skip_count += 1
        
        print(f"  ✓ Successfully merged {org_merge_count} organization node(s)")
        if org_skip_count > 0:
            print(f"  ⚠ Skipped {org_skip_count} invalid merge(s)")
        
        # AFTER stats
        print(f"\n[4/4] Capturing AFTER statistics...")
        self.stats_after = GraphStats.get_stats(self.driver)
        GraphStats.print_stats(self.stats_after, "AFTER ENTITY RESOLUTION")
        
        # Comparison
        GraphStats.print_comparison(self.stats_before, self.stats_after)
        
        elapsed = time.time() - start_time
        print(f"\n⏱️  Entity Resolution completed in {elapsed:.2f} seconds")
        print("=" * 70)


# =====================================================================
# MAIN PIPELINE
# =====================================================================

def main():
    # Configuration
    JSONL_PATH = "/qwen_output_v1.jsonl"
    NEO4J_URI = "neo4j://127.0.0.1:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PW = "your_password"
    
    print("=" * 70)
    print("KNOWLEDGE GRAPH IMPORT & ENTITY RESOLUTION PIPELINE (FIXED)")
    print("=" * 70)
    
    # STEP 1: Import all data (creates duplicates)
    print("\n[STEP 1/3] Importing JSONL data...")
    import_jsonl_to_neo4j(
        jsonl_path=JSONL_PATH,
        uri=NEO4J_URI,
        user=NEO4J_USER,
        password=NEO4J_PW,
        log_every=50
    )
    
    # STEP 2: Preview duplicates (optional - shows what will be merged)
    print("\n[STEP 2/3] Previewing potential duplicates...")
    resolver = EntityResolver(NEO4J_URI, NEO4J_USER, NEO4J_PW)
    try:
        resolver.preview_duplicates(
            person_threshold=0.95,  # Conservative threshold
            org_threshold=0.90
        )
        
        # STEP 3: Entity resolution (merges duplicates) automatically
        print("\n[STEP 3/3] Running entity resolution...")
        resolver.resolve_all(
            person_threshold=0.95,  # High threshold = only merge very similar
            org_threshold=0.90
        )
        
    finally:
        resolver.close()
    
    print("\n" + "=" * 70)
    print("PIPELINE COMPLETE!")
    print("=" * 70)


if __name__ == "__main__":
    main()

KNOWLEDGE GRAPH IMPORT & ENTITY RESOLUTION PIPELINE (FIXED)

[STEP 1/3] Importing JSONL data...
[INFO] Processing line 50... (success=49, failed=0, skipped=0)
	 took 2.4175379276275635 seconds
[INFO] Processing line 100... (success=99, failed=0, skipped=0)
	 took 5.129435777664185 seconds
[INFO] Processing line 150... (success=149, failed=0, skipped=0)
	 took 8.505815982818604 seconds
[INFO] Processing line 200... (success=199, failed=0, skipped=0)
	 took 14.655655860900879 seconds
[INFO] Processing line 250... (success=249, failed=0, skipped=0)
	 took 18.87693476676941 seconds
[INFO] Processing line 300... (success=299, failed=0, skipped=0)
	 took 24.275864839553833 seconds
[INFO] Processing line 350... (success=349, failed=0, skipped=0)
	 took 29.374151945114136 seconds
[INFO] Processing line 400... (success=399, failed=0, skipped=0)
	 took 35.32025980949402 seconds
[INFO] Processing line 450... (success=449, failed=0, skipped=0)
	 took 42.1365327835083 seconds
[INFO] Processing line

/opt/anaconda3/envs/MAS_200/lib/python3.11/site-packages/neo4j/_sync/work/result.py:635: UserWarning: Expected a result with a single record, but found multiple.
  warn(


    Merging 'the american geriatrics society' -> 'american geriatrics society' (similarity: 0.931)
    Merging 'teva pharmaceuticals' -> 'teva pharmaceutical' (similarity: 0.974)
    Merging 'healthcare distribution alliance hda' -> 'healthcare distribution alliance' (similarity: 0.941)
    Merging 'norwalk2' -> 'norwalk' (similarity: 0.933)
    Merging 'national association of boards of pharmacy nabp' -> 'national association of boards of pharmacy' (similarity: 0.944)
    Merging 'teva pharmaceuticals usa' -> 'teva pharmaceuticals' (similarity: 0.909)
  ✓ Successfully merged 23 organization node(s)
  ⚠ Skipped 27 invalid merge(s)

[4/4] Capturing AFTER statistics...

                       AFTER ENTITY RESOLUTION                        

📊 OVERALL:
  Total Nodes:         16,987
  Total Relationships: 30,105

📦 NODES BY TYPE:
  Person               4,036
  TopicEntity          2,659
  Email                2,217
  Event                1,924
  Concern              1,802
  Decision       